# COVID 19 API

In [4]:
import requests
import pandas as pd
import json

# India Cases and Deaths

In [15]:

# Make a GET request to the COVID-19 API endpoint for daily deaths and positive cases data
url = "https://api.covid19api.com/country/india?from=2020-01-01T00:00:00Z&to=2023-01-01T00:00:00Z"
response = requests.get(url)

# Parse the JSON response and extract the daily deaths and positive cases data
data = json.loads(response.text)
dates = [d['Date'] for d in data]
deaths = [d['Deaths'] for d in data]
cases = [d['Confirmed'] for d in data]

# Create a pandas dataframe to store the daily deaths and positive cases data
india_df = pd.DataFrame({'Date': dates, 'Deaths': deaths, 'Cases': cases})

india_df['Date'] = pd.to_datetime(india_df['Date']).dt.date
india_df['Date'] = india_df['Date'].astype(str)

# Display the first 5 rows of the dataframe
india_df



JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# India Vaccinations

Data source: https://github.com/owid/covid-19-data

In [14]:

# Open the JSON file and load the data
with open('owid-covid-data.json', 'r') as f:
    data = json.load(f)


# Extract the data for "IND"
india_data = data['IND']['data']

# Extract the "date" and "total_vaccinations" data for "IND"
india_vaccination_data = [(entry['date'], entry['total_vaccinations']) for entry in india_data if 'total_vaccinations' in entry]

# Create a Pandas DataFrame
vaccinations_df = pd.DataFrame(india_vaccination_data, columns=['Date', 'Total Vaccinations'])

# Convert the "total vaccinations" column to a standard decimal format
pd.options.display.float_format = '{:.2f}'.format
vaccinations_df['Total Vaccinations'] = vaccinations_df['Total Vaccinations'].astype(float)

vaccinations_df['Date'] = vaccinations_df['Date'].astype(str)



#Dates that I'm missing that I will need to manually add
#(2020, 1, 12), (2020, 2, 12), (2020, 11, 17), (2020, 11, 18), (2020, 11, 19), (2020, 12, 1), (2020, 12, 2), (2020, 12, 6), (2020, 12, 7), (2020, 12, 9), (2020, 12, 12), (2020, 12, 16), (2020, 12, 19), (2020, 12, 20), (2020, 12, 23), (2020, 12, 31), (2021, 1, 2), (2021, 1, 4), (2021, 1, 6), (2021, 1, 7), (2021, 1, 9)


# create a new dataframe with dates present in Pixstory dataset but not in my vaccinations dataset
additional_dates = [('2020', '1', '12'), ('2020', '2', '12'), ('2020', '11', '17'), ('2020', '11', '18'), ('2020', '11', '19'),
             ('2020', '12', '2'), ('2020', '12', '6'), ('2020', '12', '9'), ('2020', '12', '12'), ('2020', '12', '16'),
             ('2020', '12', '19'), ('2020', '12', '20'), ('2020', '12', '23'), ('2020', '12', '31'), ('2021', '1', '2'),
             ('2021', '1', '6'), ('2021', '1', '7'), ('2021', '1', '9')]

additional_dates_df = pd.DataFrame(additional_dates, columns=['Year', 'Month', 'Day'])
additional_dates_df['Date'] = pd.to_datetime(additional_dates_df[['Year', 'Month', 'Day']]).dt.date

# concatenate the two dataframes
final_vaccinations_df = pd.concat([additional_dates_df, vaccinations_df], ignore_index=True)


final_vaccinations_df= final_vaccinations_df.drop(['Year', 'Month', 'Day'], axis=1)

# replace NaN with zero
final_vaccinations_df.fillna(0, inplace=True)

# Print the DataFrame
final_vaccinations_df.head(30)







,Date,Total Vaccinations
0,2020-01-12,0.00
1,2020-02-12,0.00
2,2020-11-17,0.00
3,2020-11-18,0.00
4,2020-11-19,0.00
5,2020-12-02,0.00
6,2020-12-06,0.00
7,2020-12-09,0.00
8,2020-12-12,0.00
9,2020-12-16,0.00


# Merge


In [100]:
#Merging the DataFrames on the 'Date' column

merged_df = pd.merge(india_df, vaccinations_df, on='Date')

merged_df['New COVID Cases'] = merged_df['Cases'].diff().fillna(0).astype(int)
merged_df['New Deaths'] = merged_df['Deaths'].diff().fillna(0).astype(int)
merged_df['New Vaccinations'] = merged_df['Total Vaccinations'].diff().fillna(0).astype(int)

#Dropping columns
final_covid_df = merged_df.drop(['Cases', 'Deaths', 'Total Vaccinations'], axis=1)

final_covid_df

,Date,New COVID Cases,New Deaths,New Vaccinations
0,2021-01-15,0,0,0
1,2021-01-16,15144,181,191181
2,2021-01-17,13788,145,33120
3,2021-01-18,10050,137,229748
4,2021-01-19,13816,162,220786
...,...,...,...,...
690,2022-12-28,268,2,90596
691,2022-12-29,243,1,95882
692,2022-12-30,226,3,84200
693,2022-12-31,265,3,88286


# Visualizations 

In [1]:
import matplotlib.pyplot as plt

# Set the figure size
plt.figure(figsize=(10, 6))

# Plot the three columns against the date column
#plt.plot(merged_df['Date'], merged_df['New COVID Cases'], label='Column 1')
#plt.plot(merged_df['Date'], merged_df['New Deaths'], label='Column 2')
plt.plot(merged_df['Date'], merged_df['Total Vaccinations'], label='Column 3')

# Add labels and title
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Three Columns over Time')

# Add legend
plt.legend()

# Show the plot
plt.show()


NameError: name 'merged_df' is not defined

<Figure size 720x432 with 0 Axes>